### Analyzing Accuracy of FR with Samples

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import re

##### Validation

In [3]:
frdf = pd.read_csv("download_csvs/sample_facial_recog_final.csv")
frdf.head(2)

,Unnamed: 0,filename,names,counts,fileLocation,yadav,modi,gandhi,kejriwal,owaisi,...,modi_actual,gandhi_actual,kejriwal_actual,owaisi_actual,Unknown_actual,chann_actual,yogi_actual,amit_actual,shivpal_actual,notes
0,0,vP4j5223PjTGjVkr9pBBuELplmjmP7sXoW54,"['yadav', 'Unknown', 'kejriwal', 'Unknown']","{""kejriwal"": 4}",drive/MyDrive/Capstone/Videos/videos_6/vP4j522...,1,0,0,1,0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,1,pPGjB00lPjTdywNrOpZZTpyPNG11rAId3QpB,['Unknown'],"{""gandhi"": 1}",drive/MyDrive/Capstone/Videos/videos_6/pPGjB00...,0,0,0,0,0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [4]:
frdf.fillna(0, inplace=True)

In [5]:
people = "yadav modi gandhi kejriwal owaisi".split()
accuracies = []
for person in people:
    accuracies.append((frdf[person] == frdf[f"{person}_actual"]).value_counts())

trues = [calc[0] for calc in accuracies]
falses = [calc[1] for calc in accuracies]

accuracydf = pd.DataFrame({
    "face": people, 
    "positive": trues,
    "negative": falses
})

accuracydf

,face,positive,negative
0,yadav,66,9
1,modi,60,15
2,gandhi,67,8
3,kejriwal,63,12
4,owaisi,72,3


In [17]:
4.37/5

0.874

In [15]:
accuracydf["accuracy"] = accuracydf["positive"]/75
accuracydf

,face,positive,negative,accuracy
0,yadav,66,9,0.880000
1,modi,60,15,0.800000
2,gandhi,67,8,0.893333
3,kejriwal,63,12,0.840000
4,owaisi,72,3,0.960000


In [7]:
((frdf["yadav"] == 0) & (frdf["yadav_actual"] == 0)).value_counts()[0]

63

In [8]:
confusion_mats = []
for person in people:
    tn = ((frdf[person] == 0) & (frdf[f"{person}_actual"] == 0)).value_counts()[0]
    fn = ((frdf[person] == 0) & (frdf[f"{person}_actual"] == 1)).value_counts()[0]
    tp = ((frdf[person] == 1) & (frdf[f"{person}_actual"] == 1)).value_counts()[0]
    fp = ((frdf[person] == 1) & (frdf[f"{person}_actual"] == 0)).value_counts()[0]
    cm = pd.DataFrame({"Actual Positive": [tp, fp], "Actual Negative": [tn, fn]})
    confusion_mats.append(cm)

In [9]:
person="yadav"
((frdf[person] == 1) & (frdf[f"{person}_actual"] == 1))

0     False
1     False
2     False
3     False
4     False
      ...  
70    False
71    False
72    False
73    False
74    False
Length: 75, dtype: bool

In [10]:
frdf[frdf["kejriwal"] > 1]

,Unnamed: 0,filename,names,counts,fileLocation,yadav,modi,gandhi,kejriwal,owaisi,...,modi_actual,gandhi_actual,kejriwal_actual,owaisi_actual,Unknown_actual,chann_actual,yogi_actual,amit_actual,shivpal_actual,notes
48,48,019LEGGl1LHm41w3QB77sy39g5elwJSEWWn7,"['Unknown', 'kejriwal', 'modi', 'Unknown', 'Un...","{""kejriwal"": 4}",drive/MyDrive/Capstone/Videos/videos_6/019LEGG...,0,1,0,2,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0


In [11]:
np.where(frdf["yadav"] > 0, True, False)

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
        True, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False])

In [12]:
person="yadav"
tn = ((frdf[person] == 0) & (frdf[f"{person}_actual"] == 0)).value_counts()
fn = ((frdf[person] == 0) & (frdf[f"{person}_actual"] == 1)).value_counts()
tp = ((frdf[person] == 1) & (frdf[f"{person}_actual"] == 1)).value_counts()
fp = ((frdf[person] == 1) & (frdf[f"{person}_actual"] == 0)).value_counts()

In [16]:
tp

False    69
True      3
dtype: int64

In [20]:
actual=np.where(frdf[f"{person}_actual"] > 0, True, False)
actual

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True, False, False, False, False,  True,
       False,  True, False,  True, False,  True, False, False, False])

In [13]:
confusion_mats = []
people = "yadav modi gandhi kejriwal owaisi".split()

for person in people:
    actual=pd.Series(np.where(frdf[f"{person}_actual"] > 0, True, False), name=f"{person}_actual")
    predicted=pd.Series(np.where(frdf[person] > 0, True, False), name=f"{person}_predicted")
    df_confusion = pd.crosstab(actual, predicted, margins=True)
    confusion_mats.append(df_confusion)

In [30]:
(63+59+65+60+69)/(72*5)

0.8777777777777778

In [37]:
69/72

0.9583333333333334

In [14]:
confusion_mats

[yadav_predicted  False  True  All
 yadav_actual                     
 False               63     5   68
 True                 4     3    7
 All                 67     8   75,
 modi_predicted  False  True  All
 modi_actual                     
 False              53     5   58
 True                8     9   17
 All                61    14   75,
 gandhi_predicted  False  True  All
 gandhi_actual                     
 False                63     4   67
 True                  4     4    8
 All                  67     8   75,
 kejriwal_predicted  False  True  All
 kejriwal_actual                     
 False                  60     6   66
 True                    6     3    9
 All                    66     9   75,
 owaisi_predicted  False  True  All
 owaisi_actual                     
 False                66     0   66
 True                  3     6    9
 All                  69     6   75]